<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/Kalkulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anthropic IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.2 MB/s eta 0:00:00


Kod instaluje dwa pakiety Pythona:

- `anthropic` - oficjalna biblioteka kliencka firmy Anthropic służąca do komunikacji z modelami językowymi Claude poprzez API
- `IPython` - zaawansowane środowisko interaktywne dla Pythona, które dostarcza rozszerzone możliwości do pracy w notatnikach Jupyter i konsolach interaktywnych

Polecenie zaczyna się od wykrzyknika (!), co oznacza, że jest to komenda systemowa wykonywana w środowisku Jupyter Notebook. `pip install` to standardowe polecenie do instalacji pakietów Pythona.

In [2]:
from anthropic import Anthropic
import re
from google.colab import userdata

Ten kod importuje niezbędne komponenty do pracy:

`from anthropic import Anthropic` - importuje główną klasę z biblioteki Anthropic, która służy do komunikacji z modelami językowymi Claude

`import re` - importuje moduł do obsługi wyrażeń regularnych w Pythonie, który pozwala na zaawansowane operacje na tekście, jak wyszukiwanie wzorców czy wyodrębnianie fragmentów tekstu

`from kaggle_secrets import UserSecretsClient` - importuje klasę do bezpiecznego zarządzania kluczami API i innymi wrażliwymi danymi w środowisku Kaggle. UserSecretsClient pozwala na pobieranie zapisanych wcześniej kluczy bez konieczności umieszczania ich bezpośrednio w kodzie.

In [3]:
class CFG:
    model = "claude-3-haiku-20240307"
    max_tokens = 2048

`CFG` to klasa konfiguracyjna, która przechowuje stałe ustawienia:

- `model = "claude-3-haiku-20240307"` - określa konkretną wersję modelu Claude, która ma być używana. W tym przypadku jest to Claude 3 Haiku z aktualizacją z 7 marca 2024

- `max_tokens = 2048` - ustawia maksymalną długość odpowiedzi modelu, mierzoną w tokenach. Token to podstawowa jednostka tekstu dla modelu - może to być pojedyncza litera, część słowa lub całe słowo. Wartość 2048 oznacza, że model będzie generował odpowiedzi nie dłuższe niż ta liczba tokenów

Użycie klasy konfiguracyjnej pozwala na łatwe zarządzanie tymi parametrami w jednym miejscu, zamiast wpisywania ich wielokrotnie w różnych częściach kodu.

In [5]:
secret_value_0 = userdata.get('anthropic')
client = Anthropic(api_key = secret_value_0)

Ten fragment kodu konfiguruje połączenie z API Claude'a:

`user_secrets = UserSecretsClient()` - tworzy obiekt do zarządzania kluczami API w środowisku Kaggle

`secret_value_0 = user_secrets.get_secret("claudekey")` - pobiera zapisany wcześniej klucz API z bezpiecznego magazynu Kaggle, gdzie został zapisany pod nazwą "claudekey"

`client = Anthropic(api_key = secret_value_0)` - tworzy obiekt klienta API Claude'a, używając pobranego klucza. Ten obiekt będzie używany do wysyłania zapytań do modelu.

Ta metoda jest bezpieczniejsza niż wpisywanie klucza API bezpośrednio w kod, ponieważ klucz pozostaje ukryty i nie może zostać przypadkowo udostępniony.

# Functions

In [6]:
def calculate(expression):
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)

    try:
        result = eval(expression)
        return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
        return "Error: Invalid expression"

Funkcja `calculate` przetwarza i wykonuje wyrażenia matematyczne:

`expression = re.sub(r'[^0-9+\-*/().]', '', expression)` - oczyszcza wejściowe wyrażenie:
- używa wyrażenia regularnego do usunięcia wszystkich znaków oprócz:
  - cyfr (0-9)
  - operatorów matematycznych (+, -, *, /)
  - nawiasów
  - kropki dziesiętnej
- zapobiega to wstrzyknięciu złośliwego kodu

`try/except` obsługuje obliczenia:
- `eval(expression)` - wykonuje wyrażenie matematyczne i zwraca wynik
- w przypadku błędów takich jak:
  - nieprawidłowa składnia
  - dzielenie przez zero
  - błędy nazw
  - nieodpowiednie typy danych
  - przepełnienie liczb
  zwracany jest komunikat "Error: Invalid expression"

Wynik jest zawsze konwertowany na tekst przed zwróceniem.

In [7]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "calculator":
        return calculate(tool_input["expression"])


Funkcja `process_tool_call` zarządza narzędziami dostępnymi dla modelu:

`if tool_name == "calculator":` - sprawdza, czy wywołane narzędzie to kalkulator

`return calculate(tool_input["expression"])` - jeśli tak:
- pobiera wyrażenie matematyczne z parametru "expression" w przekazanych danych wejściowych
- przekazuje je do wcześniej zdefiniowanej funkcji `calculate`
- zwraca wynik obliczenia

In [8]:
def chat_with_claude(user_message):

    message = client.messages.create(
        model=CFG.model, max_tokens= CFG.max_tokens, messages=[{"role": "user", "content": user_message}],
        tools=tools,
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")

    if message.stop_reason == "tool_use":
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"Tool Result: {tool_result}")

        response = client.messages.create(
            model=CFG.model,
            max_tokens = CFG.max_tokens,
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": tool_result,
                        }
                    ],
                },
            ],
            tools=tools,
        )
    else:
        response = message

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )
    print(response.content)
    print(f"\nFinal Response: {final_response}")

    return final_response

Funkcja `chat_with_claude` zarządza pełną konwersacją z modelem Claude:

1. Pierwsze wywołanie API:
```python
message = client.messages.create(
    model=CFG.model,
    max_tokens=CFG.max_tokens,
    messages=[{"role": "user", "content": user_message}],
    tools=tools
)
```
Wysyła wiadomość użytkownika do modelu.

2. Sprawdzenie wyniku:
- Wyświetla przyczynę zakończenia i treść odpowiedzi
- Jeśli `stop_reason == "tool_use"`, oznacza to że model chce użyć narzędzia:
  - Znajduje szczegóły użycia narzędzia w odpowiedzi
  - Wywołuje odpowiednie narzędzie przez `process_tool_call`
  - Wysyła drugie zapytanie do API zawierające:
    - oryginalną wiadomość użytkownika
    - pierwszą odpowiedź modelu
    - wynik działania narzędzia
- Jeśli nie, zachowuje pierwszą odpowiedź

3. Ekstrakcja końcowej odpowiedzi:
- Szuka bloku z właściwością `text` w odpowiedzi
- Wyświetla pełną odpowiedź i jej wersję tekstową
- Zwraca końcową odpowiedź tekstową

Ta funkcja pozwala na dwustopniową interakcję, gdzie model może używać narzędzi i uwzględniać ich wyniki w swojej finalnej odpowiedzi.

# Test

In [9]:
tools = [
    {
        "name": "calculator",
        "description": "A simple calculator that performs basic arithmetic operations.",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate (e.g., '2 + 3 * 4')."
                }
            },
            "required": ["expression"]
        }
    }
]

`tools` to lista definiująca narzędzia dostępne dla modelu. Aktualnie zawiera jeden element - kalkulator:

`name`: "calculator" - nazwa narzędzia używana do jego identyfikacji

`description`: Opis informujący model, że jest to prosty kalkulator do podstawowych działań arytmetycznych

`input_schema`: Schema JSON określająca format danych wejściowych:
- `type`: "object" - dane wejściowe muszą być obiektem
- `properties`: definiuje właściwości obiektu:
  - `expression`: pole na wyrażenie matematyczne
    - `type`: "string" - musi być tekstem
    - `description`: wyjaśnia format (np. '2 + 3 * 4')
- `required`: ["expression"] - pole expression jest wymagane

Ta definicja pozwala modelowi zrozumieć jak używać kalkulatora i jakie dane może mu przekazać.

In [10]:
chat_with_claude("What is the result of 555 * 555?")


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}}

In [ ]:
chat_with_claude("Calculate (12851 - 593) * 301 + 76")


Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text='Here is the calculation:', type='text'), ToolUseBlock(id='toolu_01EBnRkrxdkbkZPLGR8WyaPt', input={'expression': '(12851 - 593) * 301 + 76'}, name='calculator', type='tool_use')]

Tool Used: calculator
Tool Input: {'expression': '(12851 - 593) * 301 + 76'}
Tool Result: 3689734
[TextBlock(text='The result of the expression (12851 - 593) * 301 + 76 is 3689734.', type='text')]

Final Response: The result of the expression (12851 - 593) * 301 + 76 is 3689734.


'The result of the expression (12851 - 593) * 301 + 76 is 3689734.'

In [ ]:
chat_with_claude("What is 15910385 divided by 193053?")


Initial Response:
Stop Reason: tool_use
Content: [ToolUseBlock(id='toolu_01TbcipGCyFADQn23yWNMwzL', input={'expression': '15910385 / 193053'}, name='calculator', type='tool_use')]

Tool Used: calculator
Tool Input: {'expression': '15910385 / 193053'}
Tool Result: 82.41459599177428
[TextBlock(text='The result of 15910385 divided by 193053 is 82.41459599177428.', type='text')]

Final Response: The result of 15910385 divided by 193053 is 82.41459599177428.


'The result of 15910385 divided by 193053 is 82.41459599177428.'